In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [13]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.utils import np_utils
from keras.optimizers import SGD 

In [3]:
# {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3,'Sad': 4, 'Surprise': 5, 'Neutral': 6}
df=pd.read_csv('/content/gdrive/My Drive/fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [5]:
print(df["Usage"].value_counts())
print(df["emotion"].value_counts())

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64


In [6]:
x_train, y_train, x_val, y_val, x_test, y_test = [],[],[],[],[],[]
num_classes=7

for index, row in df.iterrows():
  try:
    val=row['pixels'].split(" ")
    pixels = np.array(val, 'float32')
    emotion = np_utils.to_categorical(row['emotion'], num_classes)

    if 'Training' in row['Usage']:
      x_train.append(pixels)
      y_train.append(emotion)
    elif 'PublicTest' in row['Usage']:
      x_val.append(pixels)
      y_val.append(emotion)
    elif 'PrivateTest' in row['Usage']:
      x_test.append(pixels)
      y_test.append(emotion)
  except:
      print("Error occured at index : {index} and row : {row}")
      


In [7]:
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_val = np.array(x_val, 'float32')
y_val = np.array(y_val, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(28709, 2304)
(28709, 7)
(3589, 2304)
(3589, 7)
(3589, 2304)
(3589, 7)


In [8]:
x_train = x_train.reshape(x_train.shape[0], 48, 48)
x_val = x_val.reshape(x_val.shape[0], 48, 48)
x_test = x_test.reshape(x_test.shape[0], 48, 48)

x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_val = np.repeat(x_val[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)

In [9]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(28709, 48, 48, 3)
(3589, 48, 48, 3)
(3589, 48, 48, 3)


In [24]:
batch_size=256
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)

In [25]:
base_model = applications.VGG16(weights='imagenet', include_top=False)
i=0
for layer in base_model.layers:
    layer.trainable = False
    i = i+1
    print(i,layer.name)

58892288/58889256 [==============================] - 1s 0us/step
1 input_4
2 block1_conv1
3 block1_conv2
4 block1_pool
5 block2_conv1
6 block2_conv2
7 block2_pool
8 block3_conv1
9 block3_conv2
10 block3_conv3
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_pool
16 block5_conv1
17 block5_conv2
18 block5_conv3
19 block5_pool


In [27]:
x = base_model.output
x = Dense(512, activation='relu')(x)
x = GlobalAveragePooling2D()(x) 
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x)

In [28]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs=30
checkpoint = ModelCheckpoint('pre_trained_model.h5', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto')
model.fit(train_generator, steps_per_epoch=len(x_train)//batch_size, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_data=val_generator, validation_steps=len(x_val)//batch_size)


Epoch 1/30
112/112 [==============================] - 24s 216ms/step - loss: 1.7305 - accuracy: 0.3037 - val_loss: 1.6163 - val_accuracy: 0.3619
Epoch 2/30
112/112 [==============================] - 24s 213ms/step - loss: 1.6432 - accuracy: 0.3531 - val_loss: 1.5798 - val_accuracy: 0.3873
Epoch 3/30
112/112 [==============================] - 24s 211ms/step - loss: 1.6176 - accuracy: 0.3691 - val_loss: 1.5615 - val_accuracy: 0.3962
Epoch 4/30
112/112 [==============================] - 24s 212ms/step - loss: 1.5970 - accuracy: 0.3761 - val_loss: 1.5571 - val_accuracy: 0.3993
Epoch 5/30
112/112 [==============================] - 24s 212ms/step - loss: 1.5818 - accuracy: 0.3853 - val_loss: 1.5397 - val_accuracy: 0.4129
Epoch 6/30
112/112 [==============================] - 23s 209ms/step - loss: 1.5724 - accuracy: 0.3859 - val_loss: 1.5439 - val_accuracy: 0.4049
Epoch 7/30
112/112 [==============================] - 24s 212ms/step - loss: 1.5638 - accuracy: 0.3912 - val_loss: 1.5309 - val_ac

In [29]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)

In [30]:
base_model = applications.VGG16(include_top=False, weights=None)
i=0
for layer in base_model.layers:
    layer.trainable = True
    i = i+1

In [31]:
x = base_model.output
x = Dense(512, activation='relu')(x)
x = GlobalAveragePooling2D()(x) 
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x)

In [32]:
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights("pre_trained_model.h5")
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

checkpoint = ModelCheckpoint('fine_tuned_model.h5', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto')
history=model.fit(train_generator, steps_per_epoch=len(x_train)//batch_size, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_data=val_generator, validation_steps=len(x_val)//batch_size)


Epoch 1/30
112/112 [==============================] - 27s 245ms/step - loss: 1.4367 - accuracy: 0.4469 - val_loss: 1.4278 - val_accuracy: 0.4601
Epoch 2/30
112/112 [==============================] - 27s 239ms/step - loss: 1.3828 - accuracy: 0.4686 - val_loss: 1.4033 - val_accuracy: 0.4746
Epoch 3/30
112/112 [==============================] - 27s 237ms/step - loss: 1.3504 - accuracy: 0.4832 - val_loss: 1.3720 - val_accuracy: 0.4874
Epoch 4/30
112/112 [==============================] - 26s 236ms/step - loss: 1.3196 - accuracy: 0.4994 - val_loss: 1.3507 - val_accuracy: 0.4941
Epoch 5/30
112/112 [==============================] - 27s 238ms/step - loss: 1.2963 - accuracy: 0.5071 - val_loss: 1.3160 - val_accuracy: 0.5078
Epoch 6/30
112/112 [==============================] - 27s 238ms/step - loss: 1.2736 - accuracy: 0.5140 - val_loss: 1.3047 - val_accuracy: 0.5140
Epoch 7/30
112/112 [==============================] - 26s 235ms/step - loss: 1.2542 - accuracy: 0.5268 - val_loss: 1.2758 - val_ac

In [38]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
scores = model.evaluate(test_generator)

15/15 [==============================] - 0s 30ms/step - loss: 1.1016 - accuracy: 0.5829


In [39]:
print("Loss : ",scores[0])
print("Accuracy : ",scores[1])

Loss :  1.1015994548797607
Accuracy :  0.5828921794891357
